## AIR QUALITY PREDICTION USING AQI INDEX

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as stat
import pylab 
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from ydata_profiling import ProfileReport

C:\Users\shrut\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


SystemError: initialization of _internal failed without raising an exception

In [ ]:
Air_data = pd.read_csv("city_day.csv")
Air_data.head(10)

In [ ]:
Air_data.tail(5)

In [ ]:
Air_data.shape

In [ ]:
Air_data.columns

In [ ]:
Air_data['City'].unique()

In [ ]:
Air_data['AQI_Bucket'].unique()

In [ ]:
Air_data['AQI'].max()

# EDA

In [ ]:
Air_data.info()

In [ ]:
Air_data.describe()

In [ ]:
Air_data.dtypes

In [ ]:
Air_data.isnull().sum()

In [ ]:
sns.heatmap(Air_data.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
Air_data[[i for i in Air_data.columns if i not in ["AQI_Bucket","AQI"]]].hist(bins=30, figsize=(15, 10))
plt.show()

In [ ]:
Air_data['Date'].date = pd.to_datetime(Air_data['Date'])


Air_data[['Year','Month','Day']] = Air_data['Date'].date.apply(lambda x: pd.Series(x.strftime("%Y,%m,%d").split(",")))

Air_data

In [ ]:
Air_data.drop(['Date'], axis=1 ,inplace=True)

Air_data

In [ ]:
# Arranging city wise PM 2.5 levels mean values 

a =Air_data.groupby("City")["PM2.5"].mean().round()
a


In [ ]:
a.plot(figsize=(7,7),kind="barh",title="PM 2.5 count in different cities", xlabel="Cities", ylabel="PM2.5 VALUES",grid=True)

In [ ]:
# Arranging city wise PM 10 levels mean values 

b = Air_data.groupby("City")["PM10"].mean().sort_values(ascending=False)
b

In [ ]:
b.head(5)

In [ ]:
b.head(5).plot(figsize=(7,7),kind="barh",title="PM 10 Levels in top 5 cities", xlabel="City", ylabel="City",grid=True)

# City having high AQI  value 


In [ ]:
#obtaining the data of city wise mean AQI index and arranging it in descending order.

c =Air_data.groupby("City").mean()["AQI"].round().sort_values(ascending=False)

# Plotting the horizontal plot of  top 10 cities against high AQI value, 

c.plot(figsize=(7,7),kind="barh",title="AQI (Greater the index bad is the quality of AIR)", xlabel="City", ylabel="AQI",grid=True)
print(c)

In [ ]:
# Extracting the values of AQI mean values in order of Months.

d =Air_data.groupby(['Month'])["AQI"].mean().round()
d
# Plotting the horizontal plot of Months against AQI values.

d.plot(figsize=(15,7),kind="barh",title="AQI index Month wise", xlabel="Month", ylabel="AQI",grid=True)

In [ ]:
# Extracting the values of AQI mean values in order of Years.

e =Air_data.groupby(['Year'])["AQI"].mean().round()
e
# Plotting the horizontal plot of Months against AQI values.

e.plot(figsize=(15,7),kind="barh",title="AQI index Month wise", xlabel="Year", ylabel="AQI",grid=True)

In [ ]:
sns.boxplot(x=Air_data['PM2.5'])

In [ ]:
sns.boxplot(x=Air_data['PM10'])

# PREPROCESSING OF DATA :

In [ ]:
Air_data = Air_data.dropna()
Air_data

In [ ]:
Air_data.shape

In [ ]:
sns.heatmap(Air_data.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
Air_data['AQI_Bucket'].unique()

In [ ]:
Air_data['AQI_Bucket'] = Air_data['AQI_Bucket'].map({'Good': 0, 'Moderate': 1,'Satisfactory': 2,'Poor': 3,'Very Poor': 4,'Severe': 5})
Air_data

In [ ]:
one_hot_encoder = pd.get_dummies(Air_data['City'], sparse=True)
one_hot_encoder


In [ ]:
df_z_scaled = Air_data.copy()


column = ['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2',
       'O3', 'Benzene', 'Toluene', 'Xylene']
df_z_scaled[column] = (df_z_scaled[column] - df_z_scaled[column].mean()) / df_z_scaled[column].std()


display(df_z_scaled)


In [ ]:
from sklearn.preprocessing import StandardScaler

dataset_norm_test = Air_data[[i for i in Air_data.columns if i not in ["City","Date","Year","Month","Day"]]].copy()
scaler = StandardScaler()
dataset_norm_test = pd.DataFrame(scaler.fit_transform(dataset_norm_test),columns=dataset_norm_test.columns)
dataset_norm_test.hist(bins=30, figsize=(15, 10))
plt.show()

In [ ]:
cols = list(dataset_norm_test.columns.values)
cols

In [ ]:
import scipy.stats as stats

dataset_yeo_test = dataset_norm_test[[i for i in dataset_norm_test.columns if i not in ["AQI_Bucket","AQI"]]].copy()


for i in dataset_yeo_test.columns:
    yeo_t,param = stats.yeojohnson(dataset_yeo_test[i],)
    plt.hist(yeo_t,bins=25)
    plt.xlabel(i)
    plt.show()

In [ ]:
pt = PowerTransformer(method='yeo-johnson')
data = pt.fit_transform(dataset_yeo_test)
Air_dataset = pd.DataFrame(data)
Air_dataset


In [ ]:
Air_dataset.set_axis(['PM2.5','PM10','NO','NO2','NOx','NH3','CO','SO2','O3','Benzene','Toluene','Xylene'], axis='columns', inplace=True)
Air_dataset

In [ ]:
Air_dataset.hist(bins=30, figsize=(15, 10))

In [ ]:
def plot_data(yeo_t,feature):
    plt.figure(figsize=(10,6))
    plt.subplot(1,2,1)
    Air_dataset[feature].hist()
    plt.subplot(1,2,2)
    stat.probplot(Air_dataset[feature],dist='norm',plot=pylab)
    plt.show()
plot_data(Air_dataset,'PM2.5')
    

In [ ]:
def plot_data(yeo_t,feature):
    plt.figure(figsize=(10,6))
    plt.subplot(1,2,1)
    Air_dataset[feature].hist()
    plt.subplot(1,2,2)
    stat.probplot(Air_dataset[feature],dist='norm',plot=pylab)
    plt.show()
plot_data(Air_dataset,'PM10')
    

In [ ]:
import seaborn as sns

dataset_corr = Air_dataset[[i for i in Air_dataset.columns if i!="AQI_Bucket"]]

plt.figure(figsize=(10,8))
cor = dataset_corr.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

# MODEL IMPLEMENTATION

In [ ]:
x=Air_dataset[['PM2.5','PM10','NO','NO2','NOx','CO','NH3','O3','SO2','Benzene','Toluene','Xylene']]
y=Air_data['AQI']

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.25,random_state=4)

# <h2><center>Linear Regression Model</center></h2>

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
def metrics(y_true, y_pred):
    print(f'RMSE:', mean_squared_error(y_true, y_pred) ** 0.5)
    print(f'R2 Score:', r2_score(y_true, y_pred))

In [ ]:
def accuracy(y_true,train_pred):
    error = abs(y_true -train_pred)
    mape = 100 * np.mean(error/y_true)
    accuracy = 100 - mape
    return accuracy

In [ ]:
metrics(y_test,y_pred)

In [ ]:
accuracy(y_test,y_pred)

# <h2><center>RandomForest Model</center></h2>

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
RF=RandomForestRegressor(n_estimators = 100, min_samples_split=3)
RF.fit(x_train, y_train)


In [ ]:
y_pred=model.predict(x_test)

In [ ]:
metrics(y_test,y_pred)

In [ ]:
accuracy(y_test,y_pred)

# <h2><center>ExtraTreesRegressor</center></h2>

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
selection = ExtraTreesRegressor()
selection.fit(x_train,y_train)

In [ ]:
y_pred = selection.predict(x_test)

In [ ]:
metrics(y_test,y_pred)

In [ ]:
accuracy(y_test,y_pred)

#  <h2><center>Output</center></h2>

In [ ]:
# Taking Input from the User

a=float(input("PM2.5 : "))
b=float(input("PM10 : "))
c=float(input("NO : "))
d=float(input('NO2 : '))
e=float(input('NOx : '))
f=float(input('CO : '))
g=float(input('NH3 : '))
h=float(input('O3 : '))
i=float(input('SO2 : '))
j=float(input('Benzene : '))
k=float(input('Toluene : '))
l=float(input('Xylene : '))

print("="*50)


val = RF.predict([[a,b,c,d,e,f,g,h,i,j,k,l]])

if val <= 50:
    print(f'The AQI Value of {val} is Good')
elif val in range(51,100):
    print(f'The AQI Value of {val} is Unhealthy')
elif val in range(101,150):
    print(f'The AQI Value of {val} is Satisfactory')
elif val in range(151,200):
    print(f'The AQI Value of {val} is Severe')
elif val in range(201,300):
    print(f'The AQI Value of {val} is Hazardous')
else:
    print(f'The AQI Value of {val} is Dangerous')